<a href="https://colab.research.google.com/github/olaviinha/SloppyButchery/blob/main/SeniorButcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Senior Butcher <font color="#999" size="3">v 0.0.4<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="4">Sloppy Butchery @</font> <a href="https://github.com/olaviinha/SloppyButchery" target="_blank"><font color="#999" size="4">Github</font></a><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="3" color="#999"><a href="https://inha.se" target="_blank"><font color="#999">O. Inha</font></a></font></font>

Senior Butcher High Precision Beat Slicer improves the precision of beat tracking with audio analysis libraries and tools such as Librosa, Essentia, or Aubio. This notebook uses [Essentia](https://github.com/MTG/essentia) for initial beat tracking. Essentia and [Librosa](https://github.com/librosa/librosa) are both used for further audio analysis. See [details in Github](https://github.com/olaviinha/SeniorButcherBeatSlicer).

### Howto
1. Input a path to an audiofile located in your Google Drive.
2. Hit <i>Runtime > Run all</i>.

<hr size="1" color="#666"/>

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

import os
from google.colab import output
force_setup = False

pip_packages = 'spleeter essentia ffmpeg-python'

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])
last_data_file = ''

output.clear()
op(c.ok, 'Setup finished.')

##------------------------------------------------------------
##
## Imports & Defs
##
##------------------------------------------------------------

import soundfile
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
nper = np.seterr(divide = 'ignore') 

output.clear()
op(c.ok, 'Setup finished.')


In [ ]:
#@title #Slice
input_audio = "" #@param {type:"string"}
#@markdown <font size="3" color="#888">Slices will be saved in the same directory, under a new subdirectory named after the input file<br><font size="4">`<path to input audio file>/<input_audio filename>/0001.wav`</font> etc.</font>

input_audio = drive_root+input_audio
output_dir = fix_path(path_dir(input_audio)+slug(basename(input_audio)))
dir_tmp = '/content/tmp/'
dir_converted = dir_tmp+'converted/'
dir_spleet = dir_tmp+'spleet/'
sr = 44100
create_dirs([dir_tmp, dir_converted, dir_spleet, output_dir])

import sys, time, soundfile, librosa, warnings
import numpy as np
import essentia
import essentia.standard
import essentia.streaming
from essentia import Pool, array
from essentia.standard import *
from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter

def get_beats(audio, sr):
  # You may use any library or tool for initial beat tracking here
  # for as long as it returns beat starting positions in seconds.
  bt = BeatTrackerMultiFeature()
  beats, _ = bt(audio)
  return beats

def get_differences(blist, round=0):
  x = list(np.array(blist.tolist()).flatten())
  xdiff = [x[n]-x[n-1] for n in range(1,len(x))]
  if round > 0:
    rounded = [ '%.2f' % el for el in xdiff ]
    return rounded
  else:
    return xdiff

def most_frequent(list):
  freq = max(set(list), key = list.count)
  return freq

def filter_durations(durations, range, decs):
  filtered_durations = []
  for duration in durations:
    duration = float(duration)
    range = float(range)
    if round(duration, decs) == range:
      filtered_durations.append(duration)
  return filtered_durations

def get_tail_peaks(audio, nudgeTime=False, wat=''):
  dur = librosa.get_duration(audio, sr=sr)
  nudged = False
  minPos = 0.96
  minPeakDis = 0.001
  th = 0.4
  pos, amp = PeakDetection(threshold=th, minPeakDistance=minPeakDis, minPosition=minPos)(audio.astype(np.float32))
  if len(pos) > 0:
    # This slice needs handling
    for i, peak in enumerate(pos):
      if amp[i] > th and nudged == False:
        nudgePeak = peak
        nudged = True
  if nudgeTime == True and nudged == True:
    return nudgePeak
  elif nudgeTime == True and nudged == False:
    return 0
  else:
    if wat == 'amp':
      return amp
    else:
      return dur-(pos*dur)

def nudge(beats, timing_track, anal_duration, real_duration, timetravel=0, return_type='time', starting_beat=0, sr=44100):
  beat_positions = []
  a = 0
  for i, beat in enumerate(beats):
    if i > starting_beat:
      # Get slice
      s_time = beat+timetravel
      e_time = anal_duration-a
      s_sample = librosa.time_to_samples(s_time, sr=sr)
      e_sample = librosa.time_to_samples(s_time+e_time, sr=sr)
      beat_timing = timing_track[s_sample:e_sample]

      # Nudge
      nudge_peak = get_tail_peaks(beat_timing, True)
      if nudge_peak > 0:
        ns_time = s_time-(anal_duration-anal_duration*nudge_peak)-a
        ns_sample = librosa.time_to_samples(ns_time, sr=sr)
      else:
        ns_time = s_time
        ns_sample = s_sample
      ns_sample = librosa.time_to_samples(ns_time-timetravel, sr=sr)
      ne_sample = librosa.time_to_samples(ns_time+(real_duration-a), sr=sr)

      # Return start:end of each beat in either 'samples' or in 'seconds'
      if return_type == 'samples':
        beat_positions.append([ns_sample, ne_sample])
      else:
        beat_positions.append([ns_time, ns_time+(real_duration-a)])
  return beat_positions

def write_slices(beat_sample_ranges, audio_track, output_dir):
  pad = 5
  for i, beat in enumerate(beat_sample_ranges):
    beat_audio = audio_track[:, beat[0]:beat[1]]
    wav_out_file = output_dir+str(i).zfill(pad)+'.wav'
    soundfile.write(wav_out_file, beat_audio.T, sr)

def waveform(input, dur=None, peaks=[], sr=44100):

  if type(input) is np.ndarray:
    data = input
  else:
    data, sr = librosa.load(input, sr=sr, duration=dur, offset=0.0)
  plt.rcParams['axes.facecolor'] = plot_bg
  fig = plt.figure(figsize=(16, 5), frameon=False)
  #ax = fig.add_axes([0, 0, 1, 1])
  #ax.axis('off')
  if len(peaks) > 0:
    for peak in peaks:
      plt.axvline(x=peak, color='r')
  #plt.plot(data, color=plot_wav)
  librosa.display.waveplot(data, sr=sr, color=plot_wav)
  plt.show()

def display_beats(beat_sample_ranges, audio_track):
  for i, beat in enumerate(beat_sample_ranges):
    waveform(audio_track[:, beat[0]:beat[1]])




start_point = time.time()
conv_point = time.time()
op(c.title, 'Processing', path_leaf(input_audio))

print('\nConvert input file...')
output = dir_converted+slug(basename(input_audio))+'.wav'
!ffmpeg {ffmpeg_q} -y -i "{input_audio}" {wav_44} "{output}"

print('Done in', round(time.time()-conv_point, 3), 'seconds.')

spleet_point = time.time()
print('\nDrum track extraction for timing...')
warnings.filterwarnings('ignore')
# separator = Separator('spleeter:4stems')
# audio_loader = AudioAdapter.default()
audio_track, sr = librosa.load(output, sr=sr, mono=False)
# audio_track, _ = audio_loader.load(output, sample_rate=sr)
# drum_track = separator.separate(audio_track.T)['drums']
# drum_track = separator.separate(audio_track)['drums']
# print( drum_track )
# separator.separate_to_file(input_audio, '/content/tmp/timing.wav')
# timing_track = librosa.to_mono(drum_track.T)
# timing_track, _ = librosa.load('/content/tmp/timing.wav', sr=sr, mono=True)
!spleeter separate -o "{dir_spleet}" -p spleeter:4stems "{input_audio}"
timing_track, _ = librosa.load(dir_spleet+basename(input_audio)+'/drums.wav', sr=sr, mono=True)
print('Done in', round(time.time()-spleet_point, 3), 'seconds.')

# Get beats
track_point = time.time()
print('\nInitial beat tracking...')
beats = get_beats(timing_track, sr)
print('Done in', round(time.time()-spleet_point, 3), 'seconds.')

# Analyze durations
duration_point = time.time()
print('\nDecomposing lossy compression of beat interval distribution...')
decs = 2
precise_durations = get_differences(beats, 0)
compressed_durations = get_differences(beats, decs)
duration_range = most_frequent(compressed_durations)
filtered_durations = filter_durations(precise_durations, duration_range, decs)
xs_beat = min(filtered_durations)
xl_beat = max(filtered_durations)+0.02
print('Done in', round(time.time()-duration_point, 3), 'seconds.')

anal_point = time.time()
print('\nShave tail peak clusters...')
nudged_beats = nudge(beats, timing_track, xl_beat, xs_beat, 0, 'samples')
print('Done in', round(time.time()-anal_point, 3), 'seconds.')



write_point = time.time()
print('\nWrite slices to WAV files...')
write_slices(nudged_beats, audio_track, output_dir)
print('Done in', round(time.time()-write_point, 3), 'seconds.')

print('\nSlices can be found from', output_dir.replace(drive_root, ''))
print('Entire process took', round(time.time()-start_point, 3), 'seconds.')

op(c.title, 'FIN')